<a href="https://colab.research.google.com/github/younesabdolmalaky/Using-image-processing-techniques-for-audio-classification/blob/main/classification_spectogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/sp/Spectrogram1.zip -d /content/Spectrogram
!unzip /content/drive/MyDrive/sp/Spectrogram2.zip -d /content/Spectrogram
!unzip /content/drive/MyDrive/sp/Spectrogram3.zip -d /content/Spectrogram

Streaming output truncated to the last 5000 lines.
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9231.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9253.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9957.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/8935.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9526.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/8943.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9866.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9368.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9743.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_

move all folders to Spectrogram

In [3]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
import glob
from tensorflow import keras
from keras.layers.core import Dense 
from tensorflow.keras.layers import Input
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras import callbacks
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

%matplotlib inline

In [4]:
model_conv = InceptionV3(weights='imagenet', input_shape=(150, 150, 3), include_top=False)


87910968/87910968 [==============================] - 0s 0us/step


In [5]:
model_conv.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [7]:
def create_model():
  model_conv.trainable = True
  model = models.Sequential()
  model.add(model_conv)
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'binary_accuracy'])
  return model

  
  

In [8]:
def load_data(path): 
  batch_size = 128
  img_height = 150 
  img_width = 150
  data_gen = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2) # set validation split

  train_generator = data_gen.flow_from_directory(
      path,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='binary',
      subset='training') # set as training data

  validation_generator = data_gen.flow_from_directory(
      path, # same directory as training data
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='binary',
      subset='validation') # set as validation data
  return train_generator ,  validation_generator


In [9]:
def fit(path , epoch):
  model = create_model()
  train_generator , validation_generator = load_data(path)
  mch = callbacks.ModelCheckpoint(path+'/fianl-model.h5' , monitor='val_loss' , mode ='min' , save_best_only=True)
  class_weights = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(train_generator.labels),y = train_generator.labels)
  class_weights = {i : class_weights[i] for i in range(2)}
  model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=epoch, validation_data=validation_generator, validation_steps=len(validation_generator) , callbacks=[mch] , class_weight = class_weights)



In [10]:
epoch = 200

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_00'
fit(path,epoch)

Found 1135 images belonging to 2 classes.
Found 283 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 39s 2s/step - loss: 1.8402 - precision: 0.7230 - recall: 0.6613 - binary_accuracy: 0.5780 - val_loss: 1.9574 - val_precision: 0.7138 - val_recall: 1.0000 - val_binary_accuracy: 0.7138
Epoch 2/200
9/9 [==============================] - 8s 852ms/step - loss: 0.6968 - precision: 0.7174 - recall: 0.2855 - binary_accuracy: 0.4106 - val_loss: 2.5925 - val_precision: 0.7138 - val_recall: 1.0000 - val_binary_accuracy: 0.7138
Epoch 3/200
9/9 [==============================] - 7s 779ms/step - loss: 0.7216 - precision: 0.7972 - recall: 0.3498 - binary_accuracy: 0.4731 - val_loss: 15.7070 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_binary_accuracy: 0.2862
Epoch 4/200
9/9 [==============================] - 10s 1s/step - loss: 0.6713 - precision: 0.7948 - recall: 0.4549 - binary_accuracy: 0.5278 - val_loss: 1.0376 - val_precision: 0.0000e+00 - val_recal

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_02'
fit(path,epoch)

Found 1101 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 21s 2s/step - loss: 2.0556 - precision_1: 0.7545 - recall_1: 0.5178 - binary_accuracy: 0.5195 - val_loss: 243.0844 - val_precision_1: 0.6667 - val_recall_1: 0.0197 - val_binary_accuracy: 0.2664
Epoch 2/200
9/9 [==============================] - 8s 846ms/step - loss: 1.4475 - precision_1: 0.7627 - recall_1: 0.6999 - binary_accuracy: 0.6176 - val_loss: 51998.3594 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_binary_accuracy: 0.2591
Epoch 3/200
9/9 [==============================] - 7s 817ms/step - loss: 1.5118 - precision_1: 0.7813 - recall_1: 0.7601 - binary_accuracy: 0.6658 - val_loss: 22112.5156 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_binary_accuracy: 0.2591
Epoch 4/200
9/9 [==============================] - 7s 747ms/step - loss: 1.2179 - precision_1: 0.7658 - recall_1: 0.8044 - binary_accuracy: 0.6739 - val_loss:

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_04'
fit(path,epoch)

Found 1106 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 24s 2s/step - loss: 0.9650 - precision_2: 0.7924 - recall_2: 0.7291 - binary_accuracy: 0.6546 - val_loss: 3.9362 - val_precision_2: 0.7816 - val_recall_2: 0.3301 - val_binary_accuracy: 0.4291
Epoch 2/200
9/9 [==============================] - 8s 906ms/step - loss: 0.6488 - precision_2: 0.8376 - recall_2: 0.8356 - binary_accuracy: 0.7559 - val_loss: 4.5045 - val_precision_2: 0.8138 - val_recall_2: 0.9757 - val_binary_accuracy: 0.8145
Epoch 3/200
9/9 [==============================] - 8s 953ms/step - loss: 0.4011 - precision_2: 0.9172 - recall_2: 0.9105 - binary_accuracy: 0.8716 - val_loss: 0.4071 - val_precision_2: 0.8640 - val_recall_2: 0.9563 - val_binary_accuracy: 0.8545
Epoch 4/200
9/9 [==============================] - 8s 821ms/step - loss: 0.1813 - precision_2: 0.9739 - recall_2: 0.9480 - binary_accuracy: 0.9421 - val_loss: 0.5498 - val_precision_2:

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_06'
fit(path,epoch)

Found 1101 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/200
1/9 [==>...........................] - ETA: 6:22 - loss: 0.8469 - precision_1: 0.7662 - recall_1: 0.6413 - binary_accuracy: 0.6016

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_00'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_02'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_04'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_06'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/id_00'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/id_02'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/id_04'
fit(path,epoch)

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/id_06'
fit(path,epoch)

In [ ]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_00'
fit(path,epoch)

In [ ]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_02'
fit(path,epoch)

In [ ]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_04'
fit(path,epoch)

In [ ]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_06'
fit(path,epoch)